In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import seaborn as sns
import rdflib as rdf
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
#return artists obtainded after DBpedia search

def get_new_artist(df, col_to_split):
    df[col_to_split] = df[col_to_split].str.replace(r'[\[\]\'\']', '')
    influence = df[col_to_split].str.split(r', ')
    influence_list = [artist for l in influence.values for artist in l]
    new_artist = pd.Series(list(set(influence_list)))
    new_artist = new_artist[new_artist != '']
    return new_artist
    
    

In [ ]:
import unicodedata

def normalize_title(title):
    return unicodedata.normalize('NFKD', title.lower()).encode('ASCII', 'ignore').decode('utf8')

### Load dataset

**Extract artist from full csv**

In [ ]:
df_mayor_styles = pd.read_csv('/root/work/datasets/train_mayors_style.csv')
df_mayor_styles.head()

In [ ]:
artist = df_mayor_styles['artist']
artist = artist.drop_duplicates()
artist.head()

In [ ]:
artist.shape

**Extract edge artist**

In [ ]:
artist_list = pd.read_csv('/root/work/datasets/Graph Data/artist-list.csv', encoding= 'utf-8-sig', names=['artist'])
artist_list.head()

In [ ]:
artist_list.shape

In [ ]:
outer_artist = artist_list['artist'][~artist_list['artist'].isin(artist)]
outer_artist.head()

In [ ]:
outer_artist.shape

In [ ]:
df_edge_artist = pd.read_csv('/root/work/datasets/Graph Data/artist-influences.csv', encoding='utf-8-sig')
df_edge_artist.head()

In [ ]:
df_edge_artist = df_edge_artist.fillna('')
influence_aritst = get_new_artist(df_edge_artist, 'Influence')
influence_aritst.head()

In [ ]:
influence_aritst.shape

In [ ]:
outer_artist = influence_aritst[~influence_aritst.isin(df_edge_artist['Artist'])]
outer_artist.head()

In [ ]:
artist = outer_artist

## Find artist influence on DBpedia

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [ ]:
def find_influence( row ):
    
    artists = []
    keywords = row.split()
    keywords. append(row)
    keywords.reverse()

    try:
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        
        for word in keywords:
            stm = """
                SELECT DISTINCT ?Aname 
                where {
                  ?A dbo:influencedBy ?B.
                  ?A a dbo:Person.
                  ?B a dbo:Person.
                  ?B foaf:name ?Bname.
                  ?A foaf:name ?Aname 
                  FILTER(regex(?Bname, \""""+ word +"""\"))
                 }"""

            sparql.setQuery(stm)
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            for result in results["results"]["bindings"]:
                artists.append(result["Aname"]["value"])
            if len(artists) != 0 :
                break
        
    except:
            print('Problem with ' + row)
    
    return artists

In [ ]:
import math

frame = 100
max_complete_frames = math.floor(len(artist)/frame)
max_complete_frames

In [ ]:
for i in np.arange(0, max_complete_frames):
    df_test =artist[i*frame : (i+1)*frame]
    df_test_influenced = df_test.apply(find_influence)
    df_to_save = pd.DataFrame({'Artist':df_test, 'Influence':df_test_influenced})
    df_to_save.to_csv('/root/work/tmp/artist-influence_'+ str(i) +'.csv', encoding= 'utf-8-sig', index=False)
    print("Iteration " + str(i) )
    
if len(artist) % frame != 0:
    df_test = artist[max_complete_frames*frame : len(artist) ]
    df_test_influenced = df_test.apply(find_influence)
    df_to_save = pd.DataFrame({'Artist':df_test, 'Influence':df_test_influenced})
    df_to_save.to_csv('/root/work/tmp/artist-influence_'+ str(max_complete_frames ) +'.csv', encoding= 'utf-8-sig', index=False)

**Join result**

In [ ]:
df_influences = pd.read_csv('/root/work/tmp/artist-influence_0.csv', encoding= 'utf-8-sig')

for i in np.arange(1, max_complete_frames+1):
    df = pd.read_csv('/root/work/tmp/artist-influence_'+ str(i) +'.csv', encoding= 'utf-8-sig')
    df_influences = pd.concat([df_influences, df], ignore_index=True)

print(len(df_influences))

In [ ]:
df_influences.head()

In [ ]:
df_influences.to_csv('/root/work/datasets/Graph Data/artist-influences.csv', index=False, encoding= 'utf-8-sig')

In [ ]:
df_total_influences = pd.read_csv('/root/work/datasets/Graph Data/artist-influences.csv', encoding='utf-8-sig')
df_total_influences.head()

In [ ]:
df_total_influences.shape

In [ ]:
df_total_influences = pd.concat([df_total_influences, df_influences])
df_total_influences.shape

In [ ]:
df_total_influences.to_csv('/root/work/datasets/Graph Data/artist-influences-dense.csv', encoding='utf-8-sig', index=False)